In [3]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps=PorterStemmer()
lm=WordNetLemmatizer()

In [37]:
df=pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t',names=['Label','Message'])
df

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [38]:
corpus=[]
for i in range(len(df)):
    words=re.sub('[^a-zA-Z]', ' ', df['Message'][i])
    words=words.lower()
    words=words.split()
    words=[ps.stem(word)for word in words if not word in set(stopwords.words('english'))]
    words=' '.join(words)
    corpus.append(words)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x=cv.fit_transform(corpus).toarray()
y=pd.get_dummies(df['Label'],drop_first=True)


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [41]:
from sklearn.naive_bayes import MultinomialNB
spam=MultinomialNB()
classifier=spam.fit(X_train,y_train)

C:\Users\DELL\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [42]:
predict=classifier.predict(X_test)
predict

array([0, 1, 0, ..., 0, 1, 0], dtype=uint8)

In [43]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
confusin=confusion_matrix(y_test,predict)
print(confusin)
print(classification_report(y_test,predict))

[[940  15]
 [  8 152]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       955
           1       0.91      0.95      0.93       160

    accuracy                           0.98      1115
   macro avg       0.95      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [44]:
accuracy=accuracy_score(y_test,predict)
accuracy

0.979372197309417